In [ ]:
# Import modules
import psycopg2
import pandas as pd
import numpy as np
from functools import reduce
from sklearn.datasets.samples_generator import make_blobs
from sklearn.cluster import KMeans
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.decomposition import PCA

In [ ]:
conn = psycopg2.connect(database='usaspending', user='postgres', password='gtown2019', host='127.0.0.1', port='5432')

In [ ]:
df = pd.read_sql_query('SELECT recipient_duns, federal_action_obligation, base_and_exercised_options_value, base_and_all_options_value, number_of_employees, annual_revenue, awarding_sub_agency_name, primary_place_of_performance_state_code, award_type_code, product_or_service_code, naics_code, extent_competed_code, type_of_set_aside_code FROM professional_services',con=conn)

In [ ]:
## Numerical data columns

df_obs = df[['recipient_duns','federal_action_obligation']]
df_obs = pd.pivot_table(df_obs,index=['recipient_duns'],values=['federal_action_obligation'],
                        aggfunc=[np.mean], fill_value = 0)

df_exercised_value = df[['recipient_duns','base_and_exercised_options_value']]
df_exercised_value = pd.pivot_table(df_exercised_value,index=['recipient_duns'],values=['base_and_exercised_options_value'],
                        aggfunc=[np.mean], fill_value = 0)

df_all_value = df[['recipient_duns','base_and_all_options_value']]
df_all_value = pd.pivot_table(df_all_value,index=['recipient_duns'],values=['base_and_all_options_value'],
                        aggfunc=[np.mean], fill_value = 0)

df_employees = df[['recipient_duns','number_of_employees']]
df_employees = pd.pivot_table(df_employees,index=['recipient_duns'],values=['number_of_employees'],
                        aggfunc=[np.mean], fill_value = 0)

df_revenue = df[['recipient_duns','annual_revenue']]
df_revenue = pd.pivot_table(df_revenue,index=['recipient_duns'],values=['annual_revenue'],
                        aggfunc=[np.mean], fill_value = 0)

In [ ]:
## Categorical data columns

df_sub_agency = df[['recipient_duns','awarding_sub_agency_name']]
df_sub_agency = pd.pivot_table(df_sub_agency, index=['recipient_duns'], columns=['awarding_sub_agency_name'], 
                        aggfunc=len, fill_value=0)

df_state = df[['recipient_duns','primary_place_of_performance_state_code']]
df_state = pd.pivot_table(df_state, index=['recipient_duns'], columns=['primary_place_of_performance_state_code'], 
                        aggfunc=len, fill_value=0)

df_award_type = df[['recipient_duns','award_type_code']]
df_award_type = pd.pivot_table(df_award_type, index=['recipient_duns'], columns=['award_type_code'], 
                        aggfunc=len, fill_value=0)

df_psc = df[['recipient_duns','product_or_service_code']]
df_psc = pd.pivot_table(df_psc, index=['recipient_duns'], columns=['product_or_service_code'], 
                        aggfunc=len, fill_value=0)

df_naics = df[['recipient_duns','naics_code']]
df_naics = pd.pivot_table(df_naics, index=['recipient_duns'], columns=['naics_code'], 
                        aggfunc=len, fill_value=0)

df_extent = df[['recipient_duns','extent_competed_code']]
df_extent = pd.pivot_table(df_extent, index=['recipient_duns'], columns=['extent_competed_code'], 
                        aggfunc=len, fill_value=0)

df_set_aside = df[['recipient_duns','type_of_set_aside_code']]
df_set_aside = pd.pivot_table(df_set_aside, index='recipient_duns', columns='type_of_set_aside_code', 
                        aggfunc=len, fill_value=0)

In [ ]:
dfs = [df_obs, df_exercised_value, df_all_value, df_employees, df_revenue, df_sub_agency, df_state, df_award_type, 
       df_psc, df_naics, df_extent, df_set_aside]

In [ ]:
df_merged = reduce(lambda  left,right: pd.merge(left,right,on=['recipient_duns'], how='outer'), dfs)
df_merged.columns.values[0] = 'federal_action_obligation'
df_merged.columns.values[1] = 'base_and_exercised_options_value'
df_merged.columns.values[2] = 'base_and_all_options_value'
df_merged.columns.values[3] = 'number_of_employees'
df_merged.columns.values[4] = 'annual_revenue'
df_merged.head()

### Clustering (not working yet)

In [ ]:
# Get all the features columns except the index
features = list(df_merged.columns)
# Get the features data
data = df_merged[features]

In [ ]:
clustering_kmeans = KMeans(n_clusters=20, precompute_distances="auto", n_jobs=-1)
data['Cluster'] = clustering_kmeans.fit_predict(data)
data.head()

In [ ]:
data['Cluster'].unique()